In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-d3e0  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

E: Unable to locate package openjdk-8-jdk-headless
--2024-03-02 17:47:59--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.226.244.21, 13.226.244.29, 13.226.244.71, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.226.244.21|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  2 14:50 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
from pyspark.ml.feature import Tokenizer, RegexTokenizer


In [ ]:
spark

***Connecting to my Bucket***

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '332396282'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
bucket = client.get_bucket(bucket_name)

for b in blobs:
    if b.name.startswith("multistream"):
        paths.append(full_path+b.name)


***Building the inverted indices and dictionaries***


Loading wiki files

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("id", "text").rdd
doc_anchor_pairs = parquetFile.select("id", "anchor_text").rdd
doc_title_pairs = parquetFile.select("id", "title").rdd

In [ ]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
# create dict of docs and title and write to bucket
doc_title = doc_title_pairs.collectAsMap()
# upload to bucket
blob = bucket.blob('final/doc_title.pkl')
blob.upload_from_string(pickle.dumps(doc_title))

In [ ]:
# save size of entire corpus
corpus_len = len(doc_title_pairs)
# upload to bucket
blob = bucket.blob('corpus_len.pkl')
blob.upload_from_string(pickle.dumps(corpus_len))

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


def tokenize(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return tokens

def tokenize_and_len(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return len(tokens)


def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [tok for tok in tokens if tok not in all_stopwords]
  count = Counter(tokens)
  res = []
  for word in count:
    res.append((word,(id,count[word])))
  return res


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  res = sorted(unsorted_pl, key= lambda tup: tup[0])
  return res


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  res_rdd = postings.map(lambda tpl: (tpl[0], len(tpl[1])))
  return res_rdd


def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  buckets_rdd = postings.map(lambda tpl: (token2bucket_id(tpl[0]),tpl))
  grouped_rdd = buckets_rdd.groupByKey()
  res_rdd = grouped_rdd.mapValues(list)
  res = res_rdd.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))

  return res

# Creating lengths dictionaries

In [ ]:
text_len_dict = doc_text_pairs.map(lambda tpl: (tpl[1],tokenize_and_len(tpl[0])))  # dict: {doc_id: len text}
text_len_dict = text_len_dict.collectAsMap()
# upload to bucket
blob = bucket.blob('final/text_len_dict.pkl')
blob.upload_from_string(pickle.dumps(text_len_dict))

In [ ]:
title_len_dict = doc_title_pairs.map(lambda tpl: (tpl[1],tokenize_and_len(tpl[0])))  # dict: {doc_id: len title}
title_len_dict = text_len_dict.collectAsMap()
# upload to bucket
blob = bucket.blob('final/title_len_dict.pkl')
blob.upload_from_string(pickle.dumps(title_len_dict))

In [ ]:
anchor_len_dict = anchor_text.map(lambda tpl: (tpl[1],tokenize_and_len(tpl[0])))  # dict: {doc_id: len anchor}
anchor_len_dict = anchor_len_dict.collectAsMap()
# upload to bucket
blob = bucket.blob('anchor_len_dict.pkl')
blob.upload_from_string(pickle.dumps(anchor_len_dict))

# Creating INDICES AND DICTIONARIES for tf idf

In [ ]:
import numpy as np

def calculate_idf(df):
  tmp = corpus_len / df
  lg = np.log10(tmp)
  return lg

def calculate_tf_idf(postings):

  def calculate_tf(tpl):
    res = []
    for id, tf in tpl[1]:
      res.append((id, (tf / anchor_len_dict[id]) * idf[tpl[0]]))
    return res


  df = calculate_df(postings)
  idf = df.map(lambda tpl: (tpl[0], calculate_idf(tpl[1]))).collectAsMap()
  tf_idf = postings.map(lambda tpl: (tpl[0], calculate_tf(tpl)))

  return idf, tf_idf


def clac_tfidf_norm(list_of_tfidf_scores):
  ret = 0
  for score in list_of_tfidf_scores:
      ret += np.power(score,2)
  return np.sqrt(ret)

# TEXT


In [ ]:

# word counts map
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered_text)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered_text).collect()


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='final/text_postings'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/final/text_postings/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [ ]:
!gsutil ls -lh $index_dst

  3.38 MiB  2024-03-02T17:48:52Z  gs://test_bucket_anchor/postings_gcp/index.pkl
TOTAL: 1 objects, 3547602 bytes (3.38 MiB)


Dictionaries

In [ ]:
idf, postings_tfidf = calculate_tf_idf(postings_filtered_text)
docs_tfidf = postings_tfidf.flatMap(lambda x: x[1]).groupByKey().mapValues(list) #return: doc id, list of tfidf scores
doc_normalization = docs_tfidf.map(lambda x:(x[0], clac_tfidf_norm(x[1]))).collectAsMap()

# upload to bucket
blob = bucket.blob('final/idf_text.pkl')
blob.upload_from_string(pickle.dumps(idf))

# upload to bucket
blob = bucket.blob('final/text_normalization.pkl')
blob.upload_from_string(pickle.dumps(doc_normalization))

# TITLE

In [ ]:
# word counts map
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
#calculate df
w2df = calculate_df(postings_title)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_InvertedIndex'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "title_InvertedIndex.pkl"
index_dst = f'gs://{bucket_name}/title_InvertedIndex/{index_src}'
!gsutil cp $index_src $index_dst

Dictionaries

In [ ]:
idf, postings_tfidf = calculate_tf_idf(postings_title)
docs_tfidf = postings_tfidf.flatMap(lambda x: x[1]).groupByKey().mapValues(list) #return: doc id, list of tfidf scores
doc_normalization = docs_tfidf.map(lambda x:(x[0], clac_tfidf_norm(x[1]))).collectAsMap()

# upload to bucket
blob = bucket.blob('final/idf_titles.pkl')
blob.upload_from_string(pickle.dumps(idf))

# upload to bucket
blob = bucket.blob('final/titles_normalization.pkl')
blob.upload_from_string(pickle.dumps(doc_normalization))

# ANCHOR

In [ ]:

anchor_text = doc_anchor_pairs.flatMap(lambda x:x[1]).reduceByKey(lambda i,j:i + ', ' + j)
anchor_text = anchor_text.map(lambda i: Row(anchor_text=i[1], id=i[0]))
# word counts map
word_counts = anchor_text.flatMap(lambda x: word_count(x[0], x[1]))
postings_anchor = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
w2df = calculate_df(postings_anchor)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_anchor).collect()


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)


In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_anchor/{index_src}'
!gsutil cp $index_src $index_dst

Dictionaries


In [ ]:
idf, postings_tfidf = calculate_tf_idf(postings_anchor)
docs_tfidf = postings_tfidf.flatMap(lambda x: x[1]).groupByKey().mapValues(list) #return: doc id, list of tfidf scores
doc_normalization = docs_tfidf.map(lambda x:(x[0], clac_tfidf_norm(x[1]))).collectAsMap()

# upload to bucket
blob = bucket.blob('idf_anchor.pkl')
blob.upload_from_string(pickle.dumps(idf))

# upload to bucket
blob = bucket.blob('anchor_normalization.pkl')
blob.upload_from_string(pickle.dumps(doc_normalization))

# BM25

In [ ]:
def calc_term_idf_bm25(df,corpus_len):
    n = corpus_len - df + 0.5
    den = df + 0.5
    ret = np.log10((n / den) + 1)
    return ret

# text

In [ ]:
text_avg_dl = np.average(list(text_len_dict.values()))
text_term_df = calculate_df(postings_filtered_text)
text_term_idf_bm25 = text_term_df.map(lambda x: (x[0], calc_term_idf_bm25(x[1], corpus_len))).collectAsMap()

# title

In [ ]:
title_avg_dl = np.average(list(title_len_dict.values()))
title_term_df = calculate_df(postings_title)
title_term_idf_bm25 = title_term_df.map(lambda x: (x[0], calc_term_idf_bm25(x[1], corpus_len))).collectAsMap()

# anchor

In [ ]:
anchor_avg_dl = np.average(list(anchor_len_dict.values()))
anchor_term_df = calculate_df(postings_anchor)
anchor_term_idf_bm25 = anchor_term_df.map(lambda x: (x[0], calc_term_idf_bm25(x[1], corpus_len))).collectAsMap()

In [ ]:
# upload to bucket
blob = bucket.blob('text_term_idf_bm25.pkl')
blob.upload_from_string(pickle.dumps(text_term_idf_bm25))

blob = bucket.blob('title_term_idf_bm25.pkl')
blob.upload_from_string(pickle.dumps(title_term_idf_bm25))

blob = bucket.blob('anchor_term_idf_bm25.pkl')
blob.upload_from_string(pickle.dumps(anchor_term_idf_bm25))

creating a dict of all 3 avg and uploading it to bucket

In [ ]:
avg_dl_dict = {'text': text_avg_dl , 'title': title_avg_dl , 'anchor':  anchor_avg_dl}
blob = bucket.blob('avg_dl_dict.pkl')
blob.upload_from_string(pickle.dumps(avg_dl_dict))

# page rank
taking the page rank from Assignment 3, convert it to dict and uploding it to bucket

In [ ]:
import gzip
import io
import csv

file_path = 'pr/part-00000-273e4666-25ce-4ecd-9c19-7834bd65494a-c000.csv.gz'
blob = bucket.blob(file_path)
blob_data = blob.download_as_string()

decompressed_data = gzip.decompress(blob_data)
decompressed_text = decompressed_data.decode('utf-8')
csv_data = io.StringIO(decompressed_text)
csv_reader = csv.reader(csv_data)

page_rank_rdd = sc.parallelize(list(csv_reader))
page_rank = page_rank_rdd.collectAsMap()

blob = bucket.blob('page_rank.pkl')
blob.upload_from_string(pickle.dumps(page_rank))